#                             ## ACP AVEC PYOPENCL ##

## Introduction

 # Choix de technologie
 
Le projet qui suit utilise PYOPENCL pour paralléliser les calculs sur la GPU, ou plutôt la iGPU de l'ordinateur.
Nous avons opté pour PYOPENCL car, ne disposant pas de carte NVIDIA, nous ne pouvions pas utiliser PYCUDA.
Après avoir récupéré le prebuilt binary correspondant au système d'exploitation de notre machine et à la version de python.Nous avons procédé à l'installation par pip install sur le prompt Anaconda. Notons que nous n'avons pas eu besoin de charger le driver opencl pour notre CPU car il est déjà intégré.

 # Choix du Sujet
 
 Nous avons choisie de coder une ACP avec pyopencl, avec pour but final de ressortir la diagonale de la matrice de variance covariance contenant les valeurs propres représentant les variances empiriques de chaque axe,de celui qui l'explique le plus à celui qui l'explique le moins.
 L'intérêt d'une ACP est de projeter des données sur une grande dimension par réduction aux 2 dimensions qui explique le mieux la donnée.


# Préparation de l'environnement

In [1]:
# Import des modules nécessaires à la réalisation du projet
import time
import pyopencl as cl
import numpy as np
from __future__ import division
from numpy import matlib 
import datetime
import pyopencl.array
import os


In [3]:
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

In [4]:
NAME = 'Intel(R) OpenCL' 
platformes = cl.get_platforms()
devs = None
for platform in platformes:
    if platform.name == NAME:
        devs = platform.get_devices()
ctx = cl.Context(devs)
queue = cl.CommandQueue(ctx)


# Préparation du jeu de données

In [5]:
# Création de la matrice a de dimension (width=25000,height=40) remplie de valeurs de façon aléatoire

width = np.int32(25000)
height = np.int32(40)
a = np.random.randint(1, 140, size=(width, height)).astype(np.float32)


In [6]:
D=np.eye(width, k=0).astype(np.float32)
D=D*1/width

In [7]:
diag=np.diag(D).astype(np.float32)

   # Mise en place de l'ACP

## Transformation de l'échantillon

 Ici nous devons transposer notre vecteur de la diagonale des poids et la matrice a de manière à avoir $a^T$, par la suite il faut transposer le vecteur des poids de manière à avoir $D^T$ pour pouvoir calculer le centre de gravité de notre nuage de points tel que $W=a^T*D$ 

In [8]:

  #Transposition du vecteur de la diagonale contenant les poids

kernel1 = """
__kernel void func1(__global float* diag, __global float* diagT, const unsigned int U,const unsigned int V) {
    unsigned int i = get_global_id(0);
    unsigned int j = get_global_id(1);
    diagT[j + V * i] = diag[i + U * j];
}

// fin de la transposition """


U = np.int32(25000) # Taille du vecteur pour la transposition
V=np.int32(1)
diag=np.diag(D).astype(np.float32)
diagT= np.zeros([ U,V ]).astype(np.float32)
mf = cl.mem_flags
diag_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=diag )
diagT_buf = cl.Buffer(ctx, mf.WRITE_ONLY, diagT.nbytes)

#Temps de calcul

prg1 = cl.Program(ctx, kernel1).build()
t1 = datetime.datetime.now()
prg1.func1(queue, diagT.shape, None, diag_buf, diagT_buf, U,V)
diagT = np.zeros([U, V]).astype(np.float32)
cl.enqueue_copy(queue, diagT, diagT_buf)
delta_t_GPU_transpo_diag = datetime.datetime.now() - t1

### PYTHON
t2 = datetime.datetime.now()
diagT_py=np.transpose(diag)
delta_t_CPU_transpo_diag = datetime.datetime.now() - t2

    #Transposition de la matrice a

### PYOPENCL
kernel2 = """ 
__kernel void func2(__global int* a, __global int* b, const unsigned int width, const unsigned int height) {
    unsigned int i = get_global_id(0);
    unsigned int j = get_global_id(1);
    b[j + width * i] = a[i + height * j];
} 

// fin de la transposition """

b = np.zeros([height, width]).astype(np.float32)
mf = cl.mem_flags
a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a )
b_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.nbytes)

#Temps de calcul

prg2 = cl.Program(ctx, kernel2).build()
t3 = datetime.datetime.now()
prg2.func2(queue, b.shape, None, a_buf, b_buf, width, height)
b = np.zeros([height, width]).astype(np.float32)
cl.enqueue_copy(queue, b, b_buf)
delta_t_GPU_transpo_a = datetime.datetime.now() - t3


### PYTHON
t4 = datetime.datetime.now()
b_py=np.transpose(a)
delta_t_CPU_transpo_a = datetime.datetime.now() - t4

## Résultats des temps de calcul

print(delta_t_GPU_transpo_diag)
print(delta_t_CPU_transpo_diag)
print(delta_t_GPU_transpo_a) 
print(delta_t_CPU_transpo_a)



C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <func1> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

0:00:00.031253
0:00:00
0:00:00.062507
0:00:00


Nous pouvons noter que la transposition de la matrice a avec la GPU côute un peu plus de temps que les autres opérations.
Maintenant calculons le centre de gravité W, qui correspond au produit $a^TD$  vecteur des moyennes de caque variables


In [9]:
 ### Multiplication de la matrice initiale transposée b et du vecteur des diagonales
    
 ## PYOPENCL

kernel3= """__kernel void gemv(const __global float* b,

                                    const __global float* diagT,

                                    uint widthnew, uint heightnew,

                                    __global float* W)
{ // Row index

    uint y = get_global_id(0);
    if (y < heightnew) {
        // Row pointer
        const __global float* row = b + y * widthnew;
        // Compute dot product  
        float dotProduct = 0;
        for (int x = 0; x < widthnew; ++x)
            dotProduct += row[x] * diagT[x];
        // Write result to global memory
        W[y] = dotProduct;
    }
}
"""

widthnew=height
heightnew=width
W = np.zeros([widthnew, 1]).astype(np.float32)
mf = cl.mem_flags
b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
diagT_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=diagT)
W_buf = cl.Buffer(ctx, mf.WRITE_ONLY, heightnew*widthnew*4)
prg3 = cl.Program(ctx, kernel3).build()

t5 = datetime.datetime.now()
prg3.gemv(queue, W.shape, None
, b_buf
, diagT_buf
, widthnew
, heightnew, W_buf)


W = np.empty([widthnew, 1]).astype(np.float32)
cl.enqueue_copy(queue, W, W_buf)
delta_t_GPU_mul_1 = datetime.datetime.now() - t5
print(delta_t_GPU_mul_1)

# PYTHON
t6 = datetime.datetime.now()
W=b.dot(diagT)
delta_t_CPU_mul_1 = datetime.datetime.now() - t6
print(delta_t_CPU_mul_1)

0:00:00.015628
0:00:00.062507


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <gemv> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compiler

Pour cette opération, nous constatons que la GPU est plus performante que le calcul sur CPU. 
Nous devons maintenant centrer notre matrice sur le centre de gravité tel que $a-W^T$, il nous faut donc transposer le vecteur de centre de gravité avant

In [10]:
    #Transposition du centre de gravité
    
#PYOPENCL    
U = np.int32(1)
V=np.int32(40)

#Nous réutilisons le même kernel que pour la transposition précédente
WT= np.zeros([ U,V ]).astype(np.float32)
mf = cl.mem_flags
W_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=W )
WT_buf = cl.Buffer(ctx, mf.WRITE_ONLY, WT.nbytes)

prg2 = cl.Program(ctx, kernel2).build()

t7 = datetime.datetime.now()
prg2.func2(queue, WT.shape, None, W_buf, WT_buf, U,V)
WT = np.zeros([U, V]).astype(np.float32)
cl.enqueue_copy(queue, WT, WT_buf)
delta_t_GPU_transpo_W = datetime.datetime.now() - t7

#PYTHON 
t8 = datetime.datetime.now()
WT_CPU=np.transpose(W)
delta_t_CPU_transpo_W = datetime.datetime.now() - t8

print(delta_t_GPU_transpo_W)
print(delta_t_CPU_transpo_W)

0:00:00.015619
0:00:00


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <func2> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

Nous remarquons que sur la transposition, une fois de plus la GPU et la CPU sont tout aussi efficace.
Maintenant que nous avons notre centre de gravité transposé, il s'agit de centrer nos variables par leur moyenne tel que $a - W^T$

In [12]:
  ###Soustraction de la matrice d'origine par le vecteur de gravité transposé ce qui donne une matrice centrée 
    
#PYOPENCL

a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a )
WT_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=WT )
Centre_buf=cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a )
Centre=np.zeros([width,height]).astype(np.float32)
prg3 = cl.Program(ctx, """
    __kernel void sub(const unsigned int size, __global float * a, __global float * WT, __global float * Centre) {

    int i = get_global_id(1); 
    int j = get_global_id(0);

    Centre[i + size * j] = 0;

    for (int k = 0; k < size; k++)
    {
       
     Centre[i + size * j] += a[k + size * i] - WT[j + size * k];

    }

    }
    """).build()

t9 = datetime.datetime.now()
prg3.sub(queue, a.shape, None,np.int32(len(a)) ,a_buf, WT_buf, Centre_buf)
cl.enqueue_copy(queue, Centre, Centre_buf)
delta_t_GPU_Centré = datetime.datetime.now() - t9


#CPU

t10 = datetime.datetime.now()
Centre_CPU=a-WT
delta_t_CPU_Centré = datetime.datetime.now() - t10

print(delta_t_GPU_Centré)
print(delta_t_CPU_Centré)

C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <sub> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, CompilerW

0:00:32.270290
0:00:00.015629


Ici nous pouvons remarquer que le temps de calcul sur la GPU est beaucoup plus important que sur la CPU.
Il s'agit maintenant de calculer la matrice de variance covariance tel que $Centre^T*D*Centre$

Il faut donc dans un premier temps transposer la matrice centrée ensuite procéder à la multiplication

## Calcul de la matrice de variance covariance

In [13]:
  ## Transposition de la matrice centrée

#PYOPENCL    
kernel4 = """ 
__kernel void func3(__global int* a, __global int* b, const unsigned int width, const unsigned int height) {
    unsigned int i = get_global_id(0);
    unsigned int j = get_global_id(1);
    b[j + width * i] = a[i + height * j];
} 

// fin de la transposition """   

Centre_T = np.zeros([height, width]).astype(np.float32)
mf = cl.mem_flags
Centre_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Centre )
Centre_T_buf = cl.Buffer(ctx, mf.WRITE_ONLY, Centre.nbytes)


prg5 = cl.Program(ctx, kernel4).build()

t11 = datetime.datetime.now()
prg5.func3(queue, Centre_T.shape, None, Centre_buf, Centre_T_buf, width, height)
Centre_T = np.zeros([height, width]).astype(np.float32)
cl.enqueue_copy(queue, Centre_T, Centre_T_buf)
delta_t_GPU_Centré_Transpo = datetime.datetime.now() - t11

#CPU
t12 = datetime.datetime.now()
Centre_T_CPU= np.transpose(Centre)
delta_t_CPU_Centré_Transpo = datetime.datetime.now() - t12

print(delta_t_GPU_Centré_Transpo)
print(delta_t_CPU_Centré_Transpo)


0:00:00.109388
0:00:00


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <func3> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

Nous constatons ici aussi un bien meilleur temps de calcul pour la CPU que la GPU.
Procédons maintenant à la multiplication pour la matrice de vairance covariance, découpons ce calcul par une multiplication centré transposé et vecteur D. Le vecteur intermédiaire est de dimension (40,1) et le résultat final, la matrice de variance covariance est une matrice carré (40,40) 

In [20]:
# Calcul de la matrice de variance covariance

#1ère partie

#PYOPENCL
#Utilisons le Kernel3 de multiplication de matrice vecteur 

widthnew=height
heightnew=width
V1 = np.zeros([widthnew, 1]).astype(np.float32)
mf = cl.mem_flags
Centre_T_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Centre_T)
diagT_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=diagT)
V1_buf = cl.Buffer(ctx, mf.WRITE_ONLY, heightnew*widthnew*4)
prg6 = cl.Program(ctx, kernel3).build()


prg6.gemv(queue, V1.shape, None
, Centre_T_buf
, diagT_buf
, widthnew
, heightnew, V1_buf)

V1 = np.empty([widthnew, 1]).astype(np.float32)
t13 = datetime.datetime.now()
cl.enqueue_copy(queue, V1, V1_buf)
delta_t_GPU_mul_2 = datetime.datetime.now() - t13


#CPU
t14 = datetime.datetime.now()
V1= Centre_T.dot(diagT)
delta_t_CPU_mul_2 = datetime.datetime.now() - t14

print(delta_t_GPU_mul_2)
print(delta_t_CPU_mul_2)

0:00:00
0:00:00


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <gemv> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compiler

Nous pouvons constater que le temps de calcul est équivalent pour la GPU et la CPU .Passons à la deuxième portion du calcul. N
ous avons un vecteur de dim (40,1),une fois multiplié par la matrice centrée (25000,40), nous obtiendrons une matrice carrée 

In [19]:
#2ème partie

#PYOPENCL

kernel5= """__kernel void gemv2(const __global float* V1,

                                    const __global float* CENTRE,

                                    uint widthnew, uint heightnew,

                                    __global float* V)
{ // Row index

    uint y = get_global_id(0);
    if (y < heightnew) {
        // Row pointer
        const __global float* row = V1 + y * widthnew;
        // Compute dot product  
        float dotProduct = 0;
        for (int x = 0; x < widthnew; ++x)
            dotProduct += row[x] * CENTRE[x];
        // Write result to global memory
        V[y] = dotProduct;
    }
}
"""
#fixons la nouvelle ligne égale à l'ancienne colonne et la nouvelle colonne à l'ancienne ligne
widthnew=height
heightnew=width
V = np.zeros([widthnew, widthnew]).astype(np.float32)
mf = cl.mem_flags
V1_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=V1)
Centre_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Centre)
V_buf = cl.Buffer(ctx, mf.WRITE_ONLY, widthnew*widthnew*4)
prg7 = cl.Program(ctx, kernel5).build()

t15 = datetime.datetime.now()
prg7.gemv2(queue, V.shape, None
, V1_buf
, Centre_buf
, widthnew
, heightnew, V_buf)


V = np.empty([widthnew, widthnew]).astype(np.float32)
cl.enqueue_copy(queue, V, V_buf)
delta_t_GPU_mul_3 = datetime.datetime.now() - t15

print(delta_t_GPU_mul_3)
print(V.shape)

# Multiplication sur La CPU
test= V1.dot(Centre)

0:00:00.015629
(40, 40)


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <gemv2> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

ValueError: shapes (40,1) and (25000,40) not aligned: 1 (dim 1) != 25000 (dim 0)

Remarquons que le calcul n'a pas pu être fait sur la CPU contrairement à la GPU

## Conclusion

In [18]:
#Nous obtenons bien avec la GPU une matrice carrée (40,40), nos variables avec un temps de calcul minime.
#Affichons la matrice de variance covariance, sa diagonale nous renseigne sur la variance empirique de chacun des X. C'est 
#l'objet de l'ACP pour décider sur quels axes (2 dimension) projeter les données.

print(V)


[[  2.00978137e+11              inf              inf ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  4.20389539e-44   0.00000000e+00   3.92363570e-44 ...,   5.89946653e-43
    3.25913779e+12   5.89946653e-43]
 [  1.40129846e-45   1.96181785e-44   4.20389539e-44 ...,   5.89946653e-43
    2.25132119e+14   5.89946653e-43]
 ..., 
 [  1.27378597e+14   5.89946653e-43   1.27379133e+14 ...,   5.89946653e-43
    1.27388797e+14   5.89946653e-43]
 [  1.27389334e+14   5.89946653e-43   1.27389871e+14 ...,   5.89946653e-43
    1.27399535e+14   5.89946653e-43]
 [  1.27400071e+14   5.89946653e-43   1.27400608e+14 ...,   5.89946653e-43
    1.27410809e+14   5.89946653e-43]]


Notons, qu'avec une opération sur CPU classique nous n'avons pas pu produire V à cause d'un problème de broadcast de la donnée.
Intéressons nous au temps de calcul total pour la GPU et la CPU


In [21]:
#GPU
Temps_GPU= delta_t_GPU_transpo_diag + delta_t_GPU_transpo_a + delta_t_GPU_mul_1+ delta_t_GPU_transpo_W + delta_t_GPU_Centré + delta_t_GPU_Centré_Transpo + delta_t_GPU_mul_2 + delta_t_GPU_mul_3

#CPU
Temps_CPU= delta_t_CPU_transpo_diag + delta_t_CPU_transpo_a + delta_t_CPU_mul_1+ delta_t_CPU_transpo_W + delta_t_CPU_Centré + delta_t_CPU_Centré_Transpo + delta_t_CPU_mul_2 
 



In [22]:
print(Temps_GPU)
print(Temps_CPU)

0:00:32.520314
0:00:00.078136


In [ ]:
Nous pouvons nous rendre compte que le temps de calcul CPU est beaucoup moins important que la GPU, il ressort clairement que 
les codes GPU nécessitent une optimisation 